# Neural ODE solvers
In this notebook, we will take a look at four examples of using neural ODE solvers. We will get familiar with `torchdiffeq` library and use it to build models for:

1. Linear system of ODEs with time-independent dynamic function
2. Non-homogeneous ODE with time-dependent dynamic function
3. MNIST classification
4. Continuous Normalizing Flow (CNF)

Have fun!



In [ ]:
import os
from tqdm.notebook import tqdm
from collections import OrderedDict

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Normalize

import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import clear_output

!pip install torchdiffeq
from torchdiffeq import odeint_adjoint
from torchdiffeq import odeint as standard_odeint

device = 'cuda'

# Utils
Some utils for plotting and loging. Don't bother, just run.

In [ ]:
plt.rcParams.update({'font.size': 15})

log_dir = './logs'
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

In [ ]:
def make_linear_plot(xs, ys, title, xlabel, ylabel, figsize=(8, 8), markers='-', markersize='10', legend_labels=None):
    plt.figure(figsize=figsize)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    if isinstance(xs, list):
        assert len(xs) == len(ys)
        for x, y, marker, label in zip(xs, ys, markers, legend_labels):
            plt.plot(x, y, marker, markersize=markersize, label=label)
            plt.legend()        
    else:
        plt.plot(xs, ys, markers, markersize=markersize)
    plt.show()

In [ ]:
def make_gif(gt, predicted, title, xlim, ylim, ts=None):
    # predicted in the shape of (n_frames, n_points, 2)
    fig, ax = plt.subplots(figsize=(8, 8))

    def animate(i):
        ax.clear()
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
        if ts is None:
            line1, = ax.plot(gt[:, 0, 0], gt[:, 0, 1], '-', markersize='10', label='GT')
            line2, = ax.plot(predicted[i, :, 0], predicted[i, :, 1], '--', markersize='10', label='Predicted')
        else:
            line1, = ax.plot(ts, gt, '-', markersize='10', label='GT')
            line2, = ax.plot(ts, predicted[i, :], '--', markersize='10', label='Predicted')
        ax.legend()
        return line1, line2 
            
    ani = FuncAnimation(fig, animate, frames=len(predicted))    
    ani.save(os.path.join(log_dir, '_'.join(title.lower().split(' ')) + '.gif'), dpi=300, writer=PillowWriter(fps=10))

# Simple ODEs

## Linear system of ODEs - time-independent dynamic function
First, we will play with some simple linear system of ODEs to get familiar with the `torchdiffeq` library.

Consider a system of ODEs:

\begin{align}
&x_1'(t) = a x_1(t) + b x_2(t) \\
&x_2'(t) = c x_1(t) + d x_2(t) 
\end{align}

It can be written in a vectorized (horizontal) version:
$$ x'(t) = x(t)A$$
where $x(t) = [x_1(t), x_2(t)]$.

Now, let's say that we have points from trajectory of $x(t)$. Our job is to create a neural network that will aproximate our dynamic function. We need to make 3 steps:
1. Create dataset. You are given ground-truth gradients, so you need to integrate over them to get the position:
$$ x(T) = x(0) + \int_0^T x'(t) dt $$
Hint: sum instead of integral is sufficient.
2. Define neural network that only uses $x$ as an argument (no time dependency).
3. Optimize it.



In [ ]:
x0 = torch.tensor([[2., 0.]]).to(device)
ts = torch.linspace(0., 10., 1000).to(device)
A_gt = torch.tensor([[-0.1, 2.0], [-2.0, -0.5]]).to(device)

In [ ]:
def generate_2d_data(x0, ts, A_gt):
    """
    Generate data for every time step in ts.
    Use ground-truth matrix A_gt.
    """
    # TODO

In [ ]:
x_gt = generate_2d_data(x0, ts, A_gt)
make_linear_plot(x_gt[:, 0, 0].cpu(), x_gt[:, 0, 1].cpu(), '2D ODE', 'x1', 'x2')

In [ ]:
class ODEFunc(nn.Module):
    def __init__(self, hid_dim=64):
        super(ODEFunc, self).__init__()
        """
        Define neural network:
        Linear(2, hid_dim) -> SiLU -> Linear(hid_dim, 2)
        """
        # TODO

    def forward(self, t, x):
        """
        Make the forward pass, don't use t at all.
        """
        # TODO

In [ ]:
torch.manual_seed(420)
odefunc_test = ODEFunc()
x = torch.randn(5, 2)
with torch.no_grad():
    out = odefunc_test(None, x)
print(out)

# expected output:
# tensor([[-0.4450,  0.0515],
#         [ 0.0216,  0.2244],
#         [-0.1507,  0.0914],
#         [ 0.0363,  0.0892],
#         [ 0.0484,  0.0241]])

In [ ]:
torch.manual_seed(420)
odefunc = ODEFunc().to(device)
optimizer = torch.optim.Adam(odefunc.parameters(), lr=1e-2)
mse_loss = torch.nn.MSELoss()

history = []
for epoch in range(301):
    """
    Calculate predicted values x_pred and compare them to
    ground-truths using MSE loss function.
    Use standard_odeint method. 
    It takes a nn.Module dynamic function, initial point, and integration times as its inputs.
    """
    x_pred = # TODO
    loss = # TODO

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    with torch.no_grad():
        history.append(x_pred[:, 0, :].detach().cpu())
        if epoch % 10 == 0:    
            make_linear_plot([x_gt[:, 0, 0].cpu(), x_pred[:, 0, 0].cpu()], 
                             [x_gt[:, 0, 1].cpu(), x_pred[:, 0, 1].cpu()], 
                             f'Epoch {epoch} MSE {loss.item():.4f}', 'x1', 'x2', 
                             markers=['-', '--'], legend_labels=['GT', 'Predicted'])
            clear_output(wait=True)

In [ ]:
make_gif(x_gt.cpu(), torch.stack(history).cpu(), 'Linear system of ODEs', [-1.5, 2], [-1.5, 2])

## Non-homogeneous ODE - time-dependent dynamic function
Now, let's bring another factor to the table - time. Let's consider an initial value problem of the form:
$$ x'(t) = e^{-t} (\sin(2t) + \cos(2t)), $$
$$ x(0) = 1. $$

We know (using some justified guessing or Wolfram) that there is an analytical solution to this problem:
$$ x(t) = \frac{1}{5} e^{-t} (8 e^t + \sin(2t) - 3 \cos(2t)). $$

Let's find out if we can create a model that learns the solution! Here are some steps that we need to follow:

1. Generate ground truth data. This time we will do it using analytical solution.
2. Create a neural network that uses both $x$ and $t$ as its arguments. We will do it using `ConcatLinear` layer.
3. Optimize the model.



In [ ]:
def generate_1d_data(t):
    return 1/5 * torch.exp(-t) * (8 * torch.exp(t) + torch.sin(2 * t) - 3 * torch.cos(2 * t))

In [ ]:
x0 = torch.tensor([[1.0]]).to(device)
ts = torch.linspace(0., 5., 1000).to(device)

x_gt = generate_1d_data(ts).unsqueeze(-1)
make_linear_plot(ts.cpu(), x_gt[:, 0].cpu(), '1D ODE', 't', 'x')

In [ ]:
class ConcatLinear(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(ConcatLinear, self).__init__()
        """
        Define a Linear layer that concatenates t and x at its input. 
        Don't use nonlinearities.
        """
        # TODO

    def forward(self, t, x):
        """
        1. Expand dimensions of scalar t, so that it matches 2nd dimension of x, e.g.:
            for x of shape (5, 2), t should be (5, 1) to enable concatenation.
        2. Concatenate x and t.
        3. Pass it through the Linear layer. 
        """
        # TODO

In [ ]:
torch.manual_seed(420)
concat_linear_test = ConcatLinear(3, 2)
x = torch.randn(5, 3)
t = torch.tensor(1)
with torch.no_grad():
    out = concat_linear_test(t, x)
print(out)

# expected output:
# tensor([[-0.4596, -1.2096],
#         [ 0.1936, -0.2271],
#         [ 0.0486,  0.4682],
#         [-0.7598, -0.8102],
#         [ 0.5184, -0.8245]])

In [ ]:
class ODEFunc(nn.Module):
    def __init__(self, hid_dim=32):
        super(ODEFunc, self).__init__()
        """
        Define neural network using 4 * ConcatLinear layers with SiLU activations. 
        Input and output are 1D.
        Don't use the activation after last layer.
        """
        # TODO

    def forward(self, t, x):
        """
        Remember to use both t and x.
        """
        # TODO

In [ ]:
torch.manual_seed(420)
odefunc_test = ODEFunc()
x = torch.randn(5, 1)
t = torch.tensor(1)
with torch.no_grad():
    out = odefunc_test(t, x)
print(out)

# expected output:
# tensor([[0.2415],
#         [0.2396],
#         [0.2399],
#         [0.2401],
#         [0.2431]])

In [ ]:
torch.manual_seed(420)
odefunc = ODEFunc().to(device)
optimizer = torch.optim.Adam(odefunc.parameters(), lr=1e-2)
mse_loss = torch.nn.MSELoss()

history = []
for i in range(351):
    """
    Calculate x_pred and MSE loss. 
    Use standard_odeint.
    """
    x_pred = # TODO
    loss = # TODO

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    
    with torch.no_grad():
        history.append(x_pred[:, 0].detach().cpu())
        if i % 10 == 0:
            make_linear_plot([ts.cpu(), ts.cpu()], [x_gt[:, 0].cpu(), x_pred[:, 0].cpu()], 
                             f'Epoch {i} MSE {loss.item():.4f}', 't', 'x', 
                             markers=['-', '--'], legend_labels=['GT', 'Predicted'])
            clear_output(wait=True)

In [ ]:
make_gif(x_gt.cpu(), torch.stack(history).cpu(), '1D ODE', [0, 5], [1, 1.9], ts=ts.cpu())

# MNIST classification
Now, let's classify MNIST digits! A classic problem tackled from a different angle.

We want to predict label $y=x(T)$ for image $x_0 = x(0)$. Let's define change of dynamics between input and output:
$$ \frac{dx(t)}{dt} = f(x(t), t, \theta)$$

Note, that dynamic function preserves the dimensionality of the input. To make the classification, we need to add one linear layer on top of it!

In [ ]:
class ConcatLinear(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(ConcatLinear, self).__init__()
        """
        copy-paste from previous section
        """
        # TODO

In [ ]:
class ResNetBlock(nn.Module):
    def __init__(self, dim):
        super(ResNetBlock, self).__init__()
        """
        Implement block of Residual Network.
        Use 2 * ConcatLinear with and SiLU activation.
        x - - - - -
        |         |
        CL + SiLU |
        |         |
        CL        |
        |         |
        + <- - - - 
        |     
        SiLU  
        """
        # TODO

    def forward(self, t, x):
        """
        Implement the forward pass.
        Don't forget to use both t and x, and add skip connection.
        Use nonlineraity at the end.
        """
        # TODO

In [ ]:
torch.manual_seed(420)
resnet_test = ResNetBlock(2)
x = torch.randn(4, 2)
t = torch.tensor(1)
with torch.no_grad():
    out = resnet_test(t, x)
print(out)

# expected output:
# tensor([[-0.2231, -0.2664],
#         [ 0.0701, -0.2782],
#         [-0.2701, -0.2697],
#         [ 0.0243, -0.2676]])

In [ ]:
class ODEFunc(nn.Module):
    def __init__(self, in_dim, hid_dim=64, n_resnet_blocks=2):
        super(ODEFunc, self).__init__()
        """
        Let's define our dynamic function but this time in more scalable way.
        Implement NN as:
        ConcatLinear -> n_resnet_blocks * ResNetBlock -> ConcatLinear
        Use SiLU ativation everywhere but after last layer.
        Note that out_dim == in_dim.
        """
        # TODO

    def forward(self, t, x):
        """
        Implement forward pass.
        """
        # TODO

In [ ]:
torch.manual_seed(420)
odefunc_test = ODEFunc(2, hid_dim=10, n_resnet_blocks=3)
x = torch.randn(3, 2)
t = torch.tensor(1)
with torch.no_grad():
    out = odefunc_test(t, x)
print(out)

# expected output:
# tensor([[0.2548, 0.3418],
#         [0.4021, 0.2814],
#         [0.3049, 0.3282]])

In [ ]:
class ODEBlock(nn.Module):
    def __init__(self, odefunc, odeint=odeint_adjoint, rtol=1e-3, atol=1e-3):
        super(ODEBlock, self).__init__()
        """
        We will use ODEBlock to wrap everything related to ODE solver.
        """
        self.odefunc = odefunc
        self.odeint = odeint
        self.rtol = rtol
        self.atol = atol
        self.integration_times = torch.tensor([0., 1.])

    def forward(self, x):
        """
        Calculate output from self.odeint (adjoint method).
        Return only the output at time t=1.
        """
        # TODO

In [ ]:
torch.manual_seed(420)
odefunc_test = ODEFunc(2, hid_dim=10, n_resnet_blocks=3)
odeblock_test = ODEBlock(odefunc_test, odeint=standard_odeint, rtol=1e-4, atol=1e-4)
x = torch.randn(3, 2)
with torch.no_grad():
    out = odeblock_test(x)
print(out)

# expected output:
# tensor([[ 1.3297, -0.4251],
#         [-1.9886,  0.3278],
#         [-0.0987,  0.5344]])

In [ ]:
class ODEClassifier(nn.Module):
    def __init__(self, in_dim, out_dim, hid_dim=64, n_resnet_blocks=2, odeint_method=odeint_adjoint, rtol=1e-3, atol=1e-3):
        super(ODEClassifier, self).__init__()
        """
        Define classifier. We need three things:
        1. ODEFunc
        2. ODEBlock
        3. Linear layer to get right dimensions for classification.
        """
        self.odefunc = # TODO
        self.ode_block = # TODO
        self.fc_out = # TODO

    def forward(self, x):
        """
        Implement forward pass through ODEBlock and Linear layer.
        """
        # TODO

In [ ]:
bsz = 2048

in_dim = 28 * 28
hid_dim = 64
out_dim = 10
n_resnet_blocks = 3

odeint_method = odeint_adjoint
rtol = 1e-3
atol = 1e-3

lr = 1e-3

n_epochs = 11

In [ ]:
data_train = MNIST('./data', train=True, download=True, transform=Compose([ToTensor(), Normalize((0.1307,), (0.3081,))]))
data_test = MNIST('./data', train=False, download=True, transform=Compose([ToTensor(), Normalize((0.1307,), (0.3081,))]))

dataloader_train = DataLoader(data_train, batch_size=bsz, shuffle=True)
dataloader_test = DataLoader(data_test, batch_size=bsz, shuffle=True)

print(f'Loaded MNIST train split with {len(data_train)} samples')
print(f'Loaded MNIST test split with {len(data_test)} samples')

In [ ]:
for i in range(3):
    x, y = data_train[i]
    plt.imshow(x[0], cmap='gray')
    plt.title(str(y))
    plt.axis('off')
    plt.show()

In [ ]:
torch.manual_seed(420)
model = ODEClassifier(in_dim, out_dim, hid_dim=hid_dim, 
                      n_resnet_blocks=n_resnet_blocks, odeint_method=odeint_adjoint, 
                      rtol=rtol, atol=atol).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
ce_loss = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(n_epochs):
    pbar = tqdm(dataloader_train, desc=f'Epoch {epoch}')
    for x, y in pbar:
        """
        Make the classification.
        Hint: use F.one_hot().
        """
        # TODO
        

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pbar.set_postfix({'CE Loss': f'{loss.item():.4f}'})
        
    if epoch % 1 == 0:
        with torch.no_grad():
            accuracy_sum = 0
            for x, y in tqdm(dataloader_test, desc='Test'):
                x = x.reshape(-1, in_dim).to(device)
                y = y.to(device)
                logits = model(x)
                accuracy_sum += (logits.argmax(1) == y).sum()
            print(f'Test accuracy after {epoch} epochs {accuracy_sum / len(data_test) * 100 :.2f}%')

# Continuous Normalizing Flow (CNF)
We will work on synthetic 2D moons dataset. Our goal is to create CNF that will learn the distribution of the data.

Let's recall the three main components of CNF:
1. Training:
$$ z_0 = z_1 + \int_1^0 f(z(t), t) dt $$
2. Loss function:
$$ \log p(z_1) = \log p(z_0) - \int_0^1 \text{tr} \bigg( \frac{f(z(t), t)}{dz(t)} \bigg) dt $$
3. Sampling:
$$ z_1 = z_0 + \int_0^1 f(z(t), t) dt $$

In [ ]:
def generate_moons(width=1.0):
    moon1 = [
        [r * np.cos(a) - 2.5, r * np.sin(a) - 1.0]
        for r in np.arange(5 - width, 5 + width, 0.1 * width)
        for a in np.arange(0, np.pi, 0.01)
    ]
    moon2 = [
        [r * np.cos(a) + 2.5, r * np.sin(a) + 1.0]
        for r in np.arange(5 - width, 5 + width, 0.1 * width)
        for a in np.arange(np.pi, 2 * np.pi, 0.01)
    ]
    points = torch.tensor(moon1 + moon2)
    points += torch.rand(points.shape) * width
    return points.float()

In [ ]:
data = generate_moons(0.5)
data = (data - data.mean(0)) / data.std(0)
print(data.shape)
plt.scatter(data[:, 0], data[:, 1], s=1)

In [ ]:
def normal_logprob(z):
	"""
	Log-probability of standard Gaussian distribution.
	"""
	return (-np.log(2 * np.pi) - 0.5 * z**2).sum(1, keepdim=True)

In [ ]:
class ConcatLinear(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(ConcatLinear, self).__init__()
        """
        copy-paste from previous section
        """
        # TODO

In [ ]:
class ResNetBlock(nn.Module):
    def __init__(self, dim):
        super(ResNetBlock, self).__init__()
        """
        copy-paste from previous section
        """
        # TODO

In [ ]:
class ODEFunc(nn.Module):
    def __init__(self, in_dim, hid_dim=64, n_resnet_blocks=2):
        super(ODEFunc, self).__init__()
        """
        copy-paste from previous section
        """
        # TODO

    def forward(self, t, states):
        """
        Implement forward pass of CNF.
        states is a tuple (z, divergence). 
        We don't need the divergence as an input
        but we need to calculate and return it for the purpose of integration.
        We have two steps:
        1. Calculate dz by passing through all of the layers.
        2. Calculate -trace(df/dz) using torch.autograd.grad().
        """
        z = states[0]

        with torch.set_grad_enabled(True):
            z.requires_grad_(True)

            dz = # TODO

            divergence = 0.
            for i in range(z.shape[1]):
                divergence += torch.autograd.grad(TODO, TODO, create_graph=True)[0][:, i]

            return dz, -divergence.reshape(-1, 1)

In [ ]:
torch.manual_seed(420)
odefunc_test = ODEFunc(2, hid_dim=10, n_resnet_blocks=3)
states = (torch.randn(3, 2), torch.randn(3, 1))
t = torch.tensor(1.)
out = odefunc_test(t, states)
print(out[0])
print(out[1])

# expected output:
# tensor([[0.2548, 0.3418],
#         [0.4021, 0.2814],
#         [0.3049, 0.3282]], grad_fn=<AddmmBackward0>)
# tensor([[0.0231],
#         [0.1463],
#         [0.0511]], grad_fn=<NegBackward0>)

In [ ]:
class CNF(nn.Module):
    def __init__(self, in_dim, hid_dim=64, n_resnet_blocks=3, odeint=odeint_adjoint, rtol=1e-3, atol=1e-3):
        super(CNF, self).__init__()
        """
        Now, let's wrap everything into CNF class.
        """
        self.odefunc = ODEFunc(in_dim, hid_dim=hid_dim, n_resnet_blocks=n_resnet_blocks)
        self.odeint = odeint
        self.rtol = rtol
        self.atol = atol

    def forward(self, z, dlogpz=None, integration_times=None, reverse=False):
        """
        Implement forward pass for CNF
        """
        # TODO
        if dlogpz is None:
            dlogpz = # TODO initialize with zeros
        if integration_times is None:
            integration_times = # TODO default integration times = [0., 1.]
        if reverse:
            integration_times = # TODO reverse the integration times
        
        states = # TODO use self.odeint pass initial states as tuple

        if len(integration_times) == 2:
            states = tuple(s[1] for s in states)
        z, dlogpz = states

        return states

In [ ]:
torch.manual_seed(420)
cnf_test = CNF(2)
z = torch.randn(3, 2)
dlogpz = torch.zeros(3, 1)

with torch.no_grad():
    z, dlogpz = cnf_test(z, dlogpz=dlogpz)
print(z)
print(dlogpz)

# expected output:
# tensor([[ 0.3435, -0.1749],
#         [-0.3217,  0.5172],
#         [ 2.0135,  0.4417]])
# tensor([[ 0.0059],
#         [ 0.0269],
#         [-0.0009]])

In [ ]:
in_dim = 2
hid_dim = 64
n_resnet_blocks = 3

odeint_method = odeint_adjoint
rtol = 1e-3
atol = 1e-3

lr = 1e-2

n_epochs = 251
n_test_samples = 10000

In [ ]:
torch.manual_seed(420)
model = CNF(in_dim, 
            hid_dim=hid_dim, 
            n_resnet_blocks=n_resnet_blocks, 
            odeint=odeint_method, 
            rtol=rtol, 
            atol=atol).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
data = torch.tensor(data).to(device)
for epoch in range(n_epochs):
    """
    Fill the blanks in the training loop.
    Calculate z0 and dlogpz using CNF's forward pass.
    Calculate logpz0 using normal_logprob().
    Calculate loss function.
    """
    z1 = data + 1e-3 * torch.randn_like(data)

    z0, dlogpz = # TODO

    logpz0 = # TODO
    logpz1 = # TODO
    loss = -torch.mean(logpz1)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Epoch {epoch} Loss {loss.item():.4f}')
    if epoch % 10 == 0:
        with torch.no_grad():
            z0 = torch.randn(n_test_samples, in_dim).to(device)
            samples = model(z0, reverse=True)[0]
            plt.figure(figsize=(5, 5))
            plt.scatter(samples[:, 0].cpu(), samples[:, 1].cpu(), alpha=0.2)
            plt.axis('off')
            plt.show()

In [ ]:
with torch.no_grad():
    integration_times = torch.arange(0., 1.01, 0.01).to(device)

    x = np.arange(-3, 3, 0.05)
    y = np.arange(-3, 3, 0.05)
    X, Y = np.meshgrid(x, y)
    X, Y = torch.tensor(X).float().to(device), torch.tensor(Y).float().to(device)
    z_grid = torch.cat([X.reshape(-1, 1), Y.reshape(-1, 1)], 1)

    dlogpz_grid = normal_logprob(z_grid)

    z1s, dlogpz1s = model(z_grid, dlogpz=dlogpz_grid, integration_times=integration_times, reverse=True)

fig, ax = plt.subplots(figsize=(8, 8))

def animate(i):
    t = integration_times[i]
    z1 = z1s[i].reshape(X.shape[0], X.shape[1], 2).cpu()
    dlogpz1 = dlogpz1s[i].reshape(X.shape).cpu()

    ax.clear()
    ax.set_xlim([-3, 3])
    ax.set_ylim([-3, 3])
    
    line = ax.pcolormesh(z1[:, :, 0], z1[:, :, 1], dlogpz1.exp())
    cmap = get_cmap(None)
    ax.set_facecolor(cmap(0.))
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])

    return line
        
ani = FuncAnimation(fig, animate, frames=len(integration_times))    
ani.save(os.path.join(log_dir, 'cnf_density.gif'), dpi=300, writer=PillowWriter(fps=10))